<table align="left">
  <td>
    <a href="https://tinyurl.com/2khrsfbc" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

# Natural Language Processing with TensorFlow

El objetivo principal del [procesamiento de lenguaje natural](https://tinyurl.com/y68tj6v6) (NLP) es analizar, comprender y derivar información valiosa de texto. Las aplicaciones de NLP pueden clasificarse en dos categorías principales:

- Texto (Como el que se encuentra en un email, un tweet, un documento, etc.)
- Audio (Como el que se encuentra en una grabación de voz, una canción, etc.)

Por ejemplo, 

- Si tuviéramos un conjunto de datos de texto que contuviera reseñas de películas, podríamos usar NLP para identificar el sentimiento detrás de cada reseña (por ejemplo, positivo o negativo). 
- También podríamos usar NLP para identificar el tema de cada reseña (por ejemplo, romance, acción, etc.). Si quisieramos construir un identificador de spam, podríamos usar NLP para identificar palabras clave en un correo electrónico que podrían indicar que es spam.
- Generación de texto: Podríamos usar NLP para generar texto, como un resumen de un artículo, un título de un artículo, una respuesta a un correo electrónico, etc.

Cuando se trabaja con texto o audio en NLP los datos tienen un orden secuencial por lo tanto el tratamiento que reciben los datos es diferente al de los datos de imágenes o tabulares.

**Aplicaciones del Modelado de Datos Secuenciales**

Al hablar de redes neuronales densas se tenía una información de entrada y se esperaba una salida sin importar el tiempo ni el orden en el que llega la información a la red. En el modelado de datos secuenciales, el orden de la información es importante:

1. En el primer ejemplo se tienen varias entradas y se obtiene una salida. 
2. En el segundo ejemplo se tiene una entrada y se quiere obtener una secuencia que represente información asociada a la acción entrante. 
3. En el ultimo ejemplo se tiene una secuencia de entrada y se espera obtener una secuencia de salida.

<a href="https://ibb.co/m0LKSmf"><img src="https://i.ibb.co/jzcYHdn/nlp-problems.png" alt="nlp-problems" border="0"></a>

Uno de los factores a tener en cuenta al trabajar con datos secuenciales, es que las redes neuronales densas no son capaces de capturar la información de secuencia. Por lo tanto, se requiere una red neuronal que sea capaz de capturar la información de secuencia. Para ello se utilizan las **redes neuronales recurrentes**.

Las **redes neuronales recurrentes (RNN**): son redes neuronales que tienen una memoria interna que les permite capturar la información de secuencia. La arquitecura base de las RNN es la siguiente:

<a href="https://ibb.co/dg8ycMy"><img src="https://i.ibb.co/9tQjvgj/arq-rnn.png" alt="arq-rnn" border="0"></a>

<a href="https://ibb.co/TrkZFZS"><img src="https://i.ibb.co/yVXDbD1/imagen-2023-02-13-105448726.png" alt="imagen-2023-02-13-105448726" border="0"></a>

>**En este notebook, se trabará sobre lo siguiente**:

- Descargar y explorar un conjunto de datos de texto
- Visualizar datos de texto
- Convertir texto en números usando tokenización
- Convertir texto tokenizado en vectores usando codificación embebida
- Modelar un dataset de texto
- Predecir el sentimiento de una oración
    - Iniciar con una referencia (TF-IDF)
    - Construir diferentes modelos de texto de deep learning
        - Dense, LSTM, GRU, Conv1D, Transfer Learning
- Comparar el desempeño de cada modelo
- Combinar los en un ensamble
- Guardar y cargar un modelo entrenado
- Encontrar las peores predicciones.

<a href="https://ibb.co/yYkdqg3"><img src="https://i.ibb.co/7RpjKr9/nlp-experiments.png" alt="nlp-experiments" border="0"></a>

In [1]:
!nvidia-smi -L

GPU 0: NVIDIA GeForce RTX 2060 with Max-Q Design (UUID: GPU-9970422a-f4b7-7ab0-4f13-419e26e877d3)


## 1. Descargar y explorar un dataset de texto

Se utilizará el dataset [Real or Not?](https://www.kaggle.com/c/nlp-getting-started/data) que contiene Tweets acerca de desastres naturales. El objetivo es predecir si un tweet es acerca de un desastre real o no.

- Un Tweet **Real** es acerca de un desastre real, por ejemplo:
> Jetstar and Virgin forced to cancel Bali flights again because of ash from Mount Raung volcano

- Un Tweet **No Real** no es acerca de un desastre real pueden ser sobre cualquier cosa, por ejemplo:
> 'Education is the most powerful weapon which you can use to change the world.' Nelson #Mandela #quote

Al descomprimir el dataset se obtendran tres archivos:

- `sample_submission.csv` - un archivo de muestra de la estructura de envío de predicciones en las competencias de Kaggle.
- `train.csv` - el conjunto de datos de entrenamiento que contiene los tweets reales y no reales de desastres etiquetados.
- `test.csv` - el conjunto de datos de prueba que contiene los tweets reales y no reales que se deben predecir.


### 1.1 Visualizar un dataset de texto

Una vez que se cuenta con un dataset para trabajar, es una buena idea visualizarlo para ver si hay algo que se pueda entender de él. El dataset se encuentra en archivos `.csv`, una forma sencilla de hacerlos legibles es usando la función `pd.read_csv()` de pandas.

In [2]:
# Turn .csv files into pandas DataFrame's
import pandas as pd
train_df = pd.read_csv("datasets/nlp_real_not_dataset/train.csv")
test_df = pd.read_csv("datasets/nlp_real_not_dataset/test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


los datos descargados seguramente se encuentran mezclados. Pero para asegurarnos es una buena idea volverlos a mezclar.

In [3]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42) # shuffle with random_state=42 for reproducibility
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


Los datos de entrenamiento contienen la columna `target` que indica si un tweet es real o no real. Esta columna será la etiqueta que se usará para entrenar el modelo. El dataset de prueba no contiene esta columna, ya que es lo que se debe predecir.

La estructura que se quiere implementar es la siguiente:

> Inputs (text column) -> Machine Learning Algorithm -> Outputs (target column)

<img src="https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/images/08-text-classification-inputs-and-outputs.png" alt="nlp" border="0" width = 800>

In [4]:
# The test data doesn't have a target (that's what we'd try to predict)
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [5]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

Dado que se cuenta con dos valores objetivo (0 y 1), se trata de un problema de clasificación binaria. El dataset se encuentra desbalanceado, con un 60% de tweets de clase 0 y un 40% de tweets de clase 1.

donde: 

- 1 = Un tweet es acerca de un desastre real
- 0 = Un tweet no es acerca de un desastre real

El dataset de entrenamiento contiene 7613 tweets, el dataset de prueba contiene 3263 tweets.

In [6]:
# How many samples total?
print(f"Total training samples: {len(train_df)}")
print(f"Total test samples: {len(test_df)}")
print(f"Total samples: {len(train_df) + len(test_df)}")

Total training samples: 7613
Total test samples: 3263
Total samples: 10876


In [7]:
# Visualización de algunos de los ejemplos de entrenamiento
import random
random_index = random.randint(0,len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
    _,text, target = row
    print(f"Target: {target}", "(real disaster)"
          if target>0 else "(not real disaster)"  
    )
    print(f"Text:\n{text}\n")
    print("---\n")


Target: 1 (real disaster)
Text:
'Among other main factors behind pedestrian fatalities are people failing to yield to a car' http://t.co/dgUL7FfJt2

---

Target: 0 (not real disaster)
Text:
What's up man?

---

Target: 0 (not real disaster)
Text:
America like South Africa is a traumatised sick country - in different ways of course - but still messed up.

---

Target: 1 (real disaster)
Text:
Cape Coral city leaders take part in mock hurricane training http://t.co/gtYCQyFuam http://t.co/qwd5PvGjbO

---

Target: 0 (not real disaster)
Text:
'If I'd have had a long coat to hand I'd have worn it. The certainty of armageddon bears a sense of occasion.'

---



### 1.2 Separar el dataset en conjuntos de entrenamiento y validación

Debido a que el dataset de prueba no contiene la columna `target`, se debe separar el dataset de entrenamiento en dos conjuntos: uno para entrenar el modelo y otro para validar el modelo. Para ello se utilizará la función [`train_test_split()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) de sklearn.

In [8]:
# split data in train and validation
from sklearn.model_selection import train_test_split
# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # dedicate 10% of samples to validation set
                                                                            random_state=42) # random state for reproducibility


In [9]:
# Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [10]:
# verificar los primeros 10 ejemplos
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

### 1.3 Convertir texto en números

Para poder alimentar texto a un modelo de deep learning, es necesario convertirlo en números. Hay varias formas de hacer esto, las más comunes son:

- **Tokenización**: Mapeado directo de palabras, caracteres o símbolos a números enteros. Hay tres niveles de tokenización:
    - **Palabra**: convertir cada palabra en una secuencia en un número entero. Con la sentencia "i love tensorflow", la palabra "i" se convertiría en el número 1, "love" en el número 2 y "tensorflow" en el número 3.
    - **Caracter**: convertir cada caracter en una secuencia en un número entero. Como convertir el abecedarios A-Z en números del 1 al 26.
    - **Subpalabra**: convertir cada subpalabra en una secuencia en un número entero. Involucra dividir las palabras en subpalabras, por ejemplo, "tensorflow" se dividiría en "ten", "sor", "flo", "w".


- **Codificación embebida**: Es una representación de lenguaje natural que puede ser aprendida. La representación se da en forma de un vector de características de longitud fija, donde cada característica representa un aspecto del lenguaje. Por ejemplo, la palabra "dance" puede ser presentada por un vector de 5 dimensiones [-0.8547, 0.4559, -0.3332, 0.9877, 0.1112]. Es importante resaltar que el tamaño del vector es ajustable. Existen dos formas de usar la codificación embebida:
    - **Codificación embebida preentrenada**: Se utiliza una codificación embebida que ya ha sido entrenada en un conjunto de datos grande. Por ejemplo, la codificación embebida preentrenada de [GloVe](https://nlp.stanford.edu/projects/glove/).
    - **Codificación embebida entrenada**: Se entrena una codificación embebida desde cero en un conjunto de datos específico. Por ejemplo, se entrena una codificación embebida en un conjunto de datos de tweets.

<a href="https://ibb.co/0sJqwkD"><img src="https://i.ibb.co/Z2TdZ3L/tokens.png" alt="tokens" border="0"></a>

>:key: **Nota**: Qué nivel de tokenización o codificación embebida se debe utilizar? 

Depende del problema que se está tratando de resolver. Es importante experimentar con diferentes niveles de tokenización y codificación embebida para ver cuál funciona mejor para tu problema.

Algunas herramientas de codificación embebida preentrenadas:

- [GloVe](https://nlp.stanford.edu/projects/glove/)
- [Word2Vec](https://jalammar.github.io/illustrated-word2vec/)

>:toolbox: **Vectorización de texto (tokenización)**

Para vectorizar el texto, se utilizará la clase [`TextVectorization`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/TextVectorization) de TensorFlow. Esta clase convierte una secuencia de caracteres en una secuencia de enteros, donde cada entero representa un token en un diccionario.

La capa `TextVectorization` toma los siguientes parametros: 

- `max_tokens` - el número máximo de tokens a tener en el diccionario. Si se establece en `None`, el diccionario tendrá tantos tokens como palabras únicas en el texto de entrenamiento.
- `standardize` - la función de normalización a aplicar a los datos de entrada. Por defecto, la función de normalización convierte el texto a minúsculas y elimina la puntuación.
- `split` - la función de separación a aplicar a los datos de entrada. Por defecto, "whitespace" separa el texto en palabras.
- `ngrams` - Cuantas palabras se deben combinar en un solo token. Por ejemplo, si `ngrams` es 2, la frase "I love TensorFlow" se convertiría en "I love", "love TensorFlow".
- `output_mode` - el modo de salida de la capa. Por defecto, "int" devuelve una secuencia de enteros (cada entero representa un token en el diccionario). `int` (mapeado entero), `binary` (one-hot encoding), `count` o `tf-idf`.
- `output_sequence_length` - Tamaño de la secuencia de salida tokenizada. Por ejemplo, si `output_sequence_length` es 10, la secuencia de salida tendrá 10 tokens, si la secuencia de entrada tiene menos de 10 tokens, se rellenará con ceros. Si la secuencia de entrada tiene más de 10 tokens, se truncará a los primeros 10 tokens.
- `pad_to_max_tokens` - Si es `True`, la secuencia de salida se rellenará con ceros hasta que tenga el tamaño de `max_tokens`.

In [11]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
# Use the default TextVectorization variables
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (all of the different words in your text)
                                    standardize="lower_and_strip_punctuation", # how to process text
                                    split="whitespace", # how to split tokens
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None) # how long should the output sequence of tokens be?
                                    # pad_to_max_tokens=True) # Not valid if using max_tokens=None

La inicialización anterior de la capa `TextVectorization` utiliza los parametros por defecto. A continuación se creará una nueva capa `TextVectorization` con los parametros que se adaptan mejor al problema.

En particular, se configurará los parametros `max_tokens` y `output_sequence_length`. 

- `max_tokens` - se establece en 10000, ya que el diccionario tendrá tantos tokens como palabras únicas en el texto de entrenamiento.
- `output_sequence_length` - se utilizará el numero promedio de tokens por tweet en el dataset de entrenamiento. El numero promedio de tokens por tweet es 15.5, por lo que se establecerá `output_sequence_length` en 15.

In [12]:
# Find average number of tokens (words) in training Tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [13]:
# Setup text vectorization with custom variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [14]:
# Ajustar el vectorizador de texto a los datos
text_vectorizer.adapt(train_sentences)

In [15]:
# Crear una sentencia de ejemplo y tokenizarla
sample_sentence = "there is a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,
          0,   0]], dtype=int64)>

In [16]:
# Escoger una muestra aleatoria del dataset de entrenamiento y tokenizarla
random_sentence = random.choice(train_sentences)
print(f"original text:\n {random_sentence}\
        \n\nVectorized version:")
text_vectorizer([random_sentence])

original text:
 #Nuclear policy of #Japan without responsibility about Nuclear #Disaster will repeat same #failure.
#annonymous #guardian #NYTimes #Reuters        

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 105,  595,    6,  224,  228, 2745,   54,  105,   75,   38, 3477,
         726,  320,    1, 1845]], dtype=int64)>

In [17]:
# obtener las palabras unicas en el vocabulario
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}") 
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


>:toolbox: **Codificación embebida**

Hasta el momento se ha convertido el texto en números enteros. Sin embargo, los números enteros no son una representación muy útil para un modelo de deep learning. Por lo tanto, se utilizará la codificación embebida para convertir los números enteros en vectores densos, que pueden ser aprendidos por el modelo.

La ventaja de la codificación embebida es que puede ser aprendida durante el entrenamiento. Por lo tanto, la codificación embebida puede capturar información específica del conjunto de datos de entrenamiento. Por ejemplo, si el conjunto de datos de entrenamiento contiene palabras como "amor", "feliz" y "triste", la codificación embebida puede aprender que "amor" y "feliz" están relacionados y "triste" no está relacionado con "amor" o "feliz".

Para codificar el texto, se utilizará la clase [`tf.keras.layers.Embedding`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding) de TensorFlow. Esta clase convierte una secuencia de enteros en una secuencia de vectores densos, donde cada entero representa un token en un diccionario.

Los parametros de la clase `Embedding` son:

- `input_dim` - el tamaño del diccionario. Por ejemplo, si el diccionario tiene 10,000 tokens, `input_dim` es 10,000. (El tamaño del diccionario se establece en el parametro `max_tokens` de la capa `TextVectorization`).
- `output_dim` - el tamaño del vector de salida. Por ejemplo, si se establece en 128, cada token se convertirá en un vector de 128 dimensiones.
- `embeddings_initializer` - la inicialización de los vectores de codificación embebida. Por defecto, los vectores de codificación embebida se inicializan con números aleatorios pequeños. Sin embargo, se puede utilizar una codificación embebida preentrenada, como [GloVe](https://nlp.stanford.edu/projects/glove/).
- `input_length` - el tamaño de la secuencia de entrada. Por ejemplo, si `input_length` es 15, la entrada debe ser una secuencia de 15 enteros.

In [18]:
tf.random.set_seed(42)
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length, # how long is each input
                             name="embedding_1") 

embedding

:key: **Nota**: Se observa que la embedding es una capa de TensorFlow, por lo que se puede utilizar como una capa de entrada en un modelo de Keras.

Ejemplo de algunas sentencias de texto y su codificación embebida:

In [19]:
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
DTN Brazil: Experts in France begin examining airplane debris found on Reunion Island: French air accident exp... http://t.co/M9IG3WQ8Lq      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-1.3418257e-02, -1.5239418e-02, -2.2546673e-02, ...,
          1.4180765e-03, -3.9529037e-02, -3.4851659e-02],
        [-3.6835194e-02, -3.2670140e-02, -1.0911129e-02, ...,
          4.2290155e-02, -3.6159564e-02,  1.8233061e-04],
        [ 3.2823417e-02,  2.2264209e-02, -2.9418886e-02, ...,
         -3.0449403e-02, -1.2066126e-02,  1.0320615e-02],
        ...,
        [ 8.6771250e-03, -1.9490719e-05, -4.0783606e-02, ...,
          1.5042376e-02,  1.5071120e-02,  8.8122115e-03],
        [-1.6325913e-02, -3.6601674e-02,  2.9070247e-02, ...,
          1.5249323e-02,  3.4499016e-02,  4.0705215e-02],
        [-2.2900343e-02,  4.0046167e-02,  7.0909038e-03, ...,
          3.6726784e-02, -4.0531863e-02, -2.5834311e-02]]], dtype=float32)>

In [20]:
# verificar un unico token en codificación embebida
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.01341826, -0.01523942, -0.02254667,  0.0494989 ,  0.00216525,
        -0.0399882 ,  0.00080553,  0.01476188,  0.03615234, -0.00884441,
         0.00621294, -0.02955726,  0.00980123,  0.01277976, -0.03157198,
         0.00294412,  0.02901384, -0.04287526, -0.04256399,  0.03591888,
        -0.04160985,  0.03295306, -0.03178471, -0.04270699, -0.01254467,
        -0.03199406,  0.00422915, -0.01716496,  0.03225711,  0.02024985,
         0.00232749,  0.04223504,  0.00964341, -0.01311073, -0.00329622,
        -0.01189204, -0.01893516,  0.00376328, -0.01839433,  0.03581586,
         0.04048808,  0.02670585,  0.02073118,  0.00023675, -0.02499862,
        -0.04086368, -0.03574042, -0.02921354,  0.00548168,  0.02125393,
         0.03762517, -0.04518021, -0.0409437 ,  0.029798  ,  0.01770559,
         0.00280501,  0.00791682, -0.03655709,  0.02362288,  0.03642707,
        -0.0153024 ,  0.00849842,  0.04856699,  0.0005394 ,  0.03691434,
  

## 2 - Modelo de deep learning para clasificación de texto

- TF-IDF: Term Frequency - Inverse Document Frequency (Frecuencia de término - Frecuencia inversa del documento). Es una medida estadística que se utiliza para evaluar la importancia de una palabra en un documento en un conjunto de documentos.

<a href="https://ibb.co/yYkdqg3"><img src="https://i.ibb.co/7RpjKr9/nlp-experiments.png" alt="nlp-experiments" border="0"></a>

Para crear los modelos se seguiran los siguientes pasos:

1. Crear un modelo de deep learning.
2. Compilar el modelo.
3. Ajustar el modelo.
4. Evaluar el modelo.


### Model 0: Naive Bayes (baseline)

Como en todos los experiments en machine learning, es importante crear un modelo de baseline para comparar con los modelos futuros.

Para crear un modelo de baseline, se utilizará el algoritmo de Naive Bayes, que es un algoritmo de clasificación de texto muy popular. Se creará un pipeline utilizando scikit-learn utilizando el metodo TF-IDF (Term Frequency - Inverse Document Frequency) para convertir el texto en números y despues de eso modelarlos utilizando el algoritmo Multinomial Naive Bayes. La selección se realizó siguiendo el mapa de algoritmos de machine learning de scikit-learn.

<img src="https://scikit-learn.org/stable/_static/ml_map.png" alt="ml-map" border="0" width=700>

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Crear tokenizacion y el pipeline del modelo
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()), # convertir las palabras en numeros utilizando tfidf
    ("clf", MultinomialNB()) # model the text
])

# Fit el pipeline a los datos
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [22]:
# Evaluar el modelo base
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Baseline Accuracy of: {baseline_score*100:0.2f}%")

Baseline Accuracy of: 79.27%


In [23]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
      dtype=int64)

In [24]:
import helper_functions as hf
baseline_results = hf.calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 1: Feed-forward neural network (dense model)

El primer modelo de deep learning que se creará será un modelo de red neuronal densa (feed-forward neural network) compuesto por una sola capa.

El modelo tomará el texto y las etiquetas como entrada, despues se tokenizara el texto, se crear un embedding, posteriormente se calculara el promedio de los embeddings (usando Global Average Pooling) y finalmente se pasara el promedio a una capa totalmente conectada con una unidad de salida y una función de activación sigmoide.

Se registran los resultados de los diferentes modelos utilizando la función `create_tensorboard_callback()` del archivo `helper_functions.py`.

In [25]:
from helper_functions import create_tensorboard_callback

# Crear directorio para salvar los logs de tensorboard

SAVE_DIR = "training_logs/RNN_text"

In [26]:
# crear un modelo con la api funcional
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string") # entradas son string de 1-dimensión
x = text_vectorizer(inputs) # convertir el texto en números
x = embedding(x) # crear un embedding del texto tokenizado
x = layers.GlobalAveragePooling1D()(x) # condensa la salida de cada token a un vector de 128, si no se utiliza, se obtiene una predicción por cada token
outputs = layers.Dense(1, activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [27]:
# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [28]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [29]:
# Fit the model
model_1_history = model_1.fit(train_sentences, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR, 
                                                                     experiment_name="simple_dense_model_1")])

Saving TensorBoard log files to: training_logs/RNN_text/simple_dense_model_1/20230225-183506
Epoch 1/5
215/215 [==============================] - 3s 8ms/step - loss: 0.6094 - accuracy: 0.6916 - val_loss: 0.5357 - val_accuracy: 0.7572
Epoch 2/5
215/215 [==============================] - 2s 7ms/step - loss: 0.4410 - accuracy: 0.8189 - val_loss: 0.4691 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 2s 7ms/step - loss: 0.3463 - accuracy: 0.8605 - val_loss: 0.4590 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 2s 7ms/step - loss: 0.2848 - accuracy: 0.8923 - val_loss: 0.4641 - val_accuracy: 0.7927
Epoch 5/5
215/215 [==============================] - 2s 7ms/step - loss: 0.2380 - accuracy: 0.9118 - val_loss: 0.4767 - val_accuracy: 0.7874


In [30]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 4ms/step - loss: 0.4767 - accuracy: 0.7874


[0.47668465971946716, 0.787401556968689]

In [31]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

(762, 1)

In [32]:
# observar las 10 primeras predicciones
model_1_pred_probs[:10]

array([[0.40488204],
       [0.74433124],
       [0.997895  ],
       [0.10889999],
       [0.11143529],
       [0.93556094],
       [0.9134595 ],
       [0.9925345 ],
       [0.97156817],
       [0.26570338]], dtype=float32)

In [33]:
# convertir las predicciones del modelo en formato de labels
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [34]:
# calcular los resultados del modelo 1
model_1_results = hf.calculate_results(y_true=val_labels, 
                                       y_pred=model_1_preds)
model_1_results

{'accuracy': 78.74015748031496,
 'precision': 0.7914920592553047,
 'recall': 0.7874015748031497,
 'f1': 0.7846966492209201}

In [35]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

>:eye: **Nota:** Se observa que el modelo de Naive Bayes tiene un accuracy de 0.79, mientras que el modelo de deep learning tiene un accuracy de 0.78. Es decir que el modelo **baseline** es mejor que el modelo de deep learning.

### Visualización de los embeddings aprendidos

In [36]:
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [37]:
# Model 1 summary
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [38]:
# Obtener la matrix de pesos de la capa embedding
# estos son los pesos que representan a cada token en los datos de entrenamiento, 
# que han sido aprendidos durante 5 epocas
embed_weights = model_1.get_layer("embedding_1").get_weights()
embed_weights

[array([[ 0.00073165,  0.01504799, -0.03425453, ..., -0.0440354 ,
         -0.01042281,  0.01876434],
        [ 0.04135861, -0.03945085, -0.03811941, ...,  0.00464737,
          0.03163553,  0.02928301],
        [ 0.00684031,  0.05363133, -0.00241554, ..., -0.07082178,
         -0.04750703,  0.01448254],
        ...,
        [-0.03301444, -0.0052493 , -0.04209725, ...,  0.02028764,
          0.00308807,  0.02215792],
        [ 0.00692343,  0.05942352, -0.01975194, ..., -0.06199061,
         -0.01018393,  0.03510419],
        [-0.0372346 ,  0.06267187, -0.07451147, ..., -0.02367218,
         -0.0864333 ,  0.01742156]], dtype=float32)]

In [39]:
# Shape tiene que tener el mismo tamaño del vocabulario y tamaño de sailda
embed_weights = model_1.get_layer("embedding_1").get_weights()[0]
print(embed_weights.shape)

(10000, 128)


Ahora tenemos la matrix de embeddings aprendidos por el modelo de deep learning. Para visualizar los embeddings, se utilizará la herramienta [Embedding Projector](https://projector.tensorflow.org/).

:key: TensorFlow tiene una guía sobre word embeddings que se puede encontrar [aquí](https://www.tensorflow.org/tutorials/text/word_embeddings).

In [40]:
# Crear embedding Files
# Code below is adapted from: https://www.tensorflow.org/tutorials/text/word_embeddings#retrieve_the_trained_word_embeddings_and_save_them_to_disk
import io

# # Create output writers
out_v = io.open("embedding_vectors.tsv", "w", encoding="utf-8")
out_m = io.open("embedding_metadata.tsv", "w", encoding="utf-8")

# # Write embedding vectors and words to file
for num, word in enumerate(words_in_vocab):
   if num == 0: 
      continue # skip padding token
   vec = embed_weights[num]
   out_m.write(word + "\n") # write words to file
   out_v.write("\t".join([str(x) for x in vec]) + "\n") # write corresponding word vector to file
out_v.close()
out_m.close()

# Download files locally to upload to Embedding Projector
try:
    from google.colab import files
except ImportError:
   pass
else:
   files.download("embeddings_projector/embedding_vectors.tsv")
   files.download("embeddings_projector/embedding_metadata.tsv")


Una vez que se han descargado los archivos, se procede a cargarlos en el [Embedding Projector](https://projector.tensorflow.org/).

1. Cargar el archivo `vectors.tsv` en el campo "Load data".
2. Cargar el archivo `metadata.tsv` en el campo "Load metadata".



## Redes Neuronales Recurrentes (RNN)

La premisa de una RNN es sencilla: usa información de la secuencia anterior para ayudar a predecir la siguiente entrada. Por ejemplo, si se está tratando de predecir la próxima palabra en una oración, la RNN puede utilizar información sobre la palabra anterior para ayudar a predecir la siguiente palabra. De esta manera, la RNN puede aprender el contexto de una secuencia de entrada. 

Este concepto es especialmente útil cuando se está trabajando con secuencias como texto, audio o video, donde el orden de las entradas es importante.

Cuando una RNN observa una secuencia de texto, los patrones que aprende se actualizan basados en la secuencia de entrada. 

Por ejemplo, observando las dos sentencias de texto a continuación:

- Massive earthquake last week, no?
- No massive earthquake last week.

Ambas frances contienen las mismas palabras pero tienen diferente significado. El orden de las palabras determina el significado de la frase.

Las RNNs se pueden utilizar para un número de problemas basados en secuencias:

- **One to one**: una entrada, una sailda, como clasificación de imágenes.
- **One to many**: una entrada, varias salidas, como subtítulado de imagenes (imagen -> texto).
- **Many to one**: varias entradas, una salida, como clasificación de texto (texto -> etiqueta).
- **Many to many**: varias entradas, varias salidas, como traducción de texto (texto -> texto) o audio a texto (audio -> texto).

<img src="http://karpathy.github.io/assets/rnn/diags.jpeg" alt="rnn" border="0" width=700>

Al trabajar con RNNs, es posible encontrar las siguientes variantes:

- LSTM (Long Short-Term Memory)
- GRU (Gated Recurrent Unit)
- Bidirectional RNNs.

:book: **Referencias**:

- [MIT Deep Learning Lecture on Recurrent Neural Networks ](https://youtu.be/SEnXr6v2ifU)
- [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)
- [Understanding LSTM Networks](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)


### Model 2: LSTM

LSTM( Long Short-Term Memory) es una variante de RNN que intenta resolver el problema del gradiente desvaneciente. Las LSTM son capaces de aprender a largo plazo y son muy útiles para tareas de procesamiento de lenguaje natural.

Las LSTM al igual que la RNN básica tienen una estructura de cadena, pero el modulo de memoria tiene una estructura diferente. En lugar de tener una sola capa de red neuronal, hay cuatro, que interactúan de una manera muy especial:

<img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png" alt="lstm" border="0" width=600>

**Notación**: En el diagrama anterior, cada línea lleva un vector entero, desde la salida de un nodo a la entrada de otro. Los circulos rosados representan una operación de pointwise (elemento a elemento), como suma de vectores, mientras que las cajas amarillas son capas entrenadas de una red neuronal. Las lineas que se fusionan denotan concatenación, mientras que una línea que se bifurca denota que su contenido se copia y las copias van a lugares distintos.

<img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM2-notation.png" alt="lstm2" border="0" width=600>

**La idea central detrás de las LSTMs**: La clave de los LSTM es el estado de la celda, la línea horizontal que atraviesa la parte superior del diagrama.

El estado de la celda es como una cinta transportadora. Se ejecuta directamente a lo largo de toda la cadena, con solo algunas interacciones lineales menores. Es muy fácil que la información fluya sin cambios.

<img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-C-line.png" alt="lstm3" border="0" width=600>

La LSTM tiene la habilidad de remover o adicionar información al estado de la celda, controlado cuidadosamente por estructuras llamadas compuertas (gates). Las compuertas son una manera de permitir que la información fluya o se bloquee en función de lo que la LSTM desea aprender. Se encuentran compuestas de una capa de red neuronal sigmoide y una multiplicación elemento a elemento.

<img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-gate.png" alt="lstm4" border="0" width=150>

La capa sigmoide genera un número entre 0 y 1, describiendo cuánto de cada unidad debe ser permitida a través. Una 1 significa "permitir completamente", mientras que una 0 significa "bloquear completamente".

Una LSTM tiene tres compuertas que controlan su estado de celda:

**Paso a paso de la LSTM:**

El primer paso en la LSTM es decidir qué información vamos a olvidar del estado de la celda anterior. Esto se hace con una compuerta de olvido, que es una capa sigmoide que aprende qué información debe ser olvidada basada en la entrada y el estado de la celda anterior.

<img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-f.png" alt="lstm5" border="0" width=600>

El siguiente paso es decidir qué nueva información se va a almacenar en el estado de la celda. Esto se hace con dos compuertas: una compuerta de entrada y una compuerta de salida. La compuerta de entrada decide que valores se actualizarán. Despues, una capa $tanh$ crea un vector de nuevos valores candidatos $\~{C}_{t}$, que se multiplican por la compuerta de entrada, para decidir qué valores se van a almacenar en el estado de la celda.

<img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-i.png" alt="lstm6" border="0" width=600>

Finalmente, es el momento de actualizarr el estado de la celda anterior, $C_{t-1}$, a la nueva celda actualizada, $C_{t}$. Primero, multiplicamos el estado de la celda anterior $f_{t}$ por la compuerta de olvido, para decidir qué parte del estado de la celda anterior se va a olvidar. Despues, sumamos el nuevo estado de la celda candidato, $i_{t}*\~{C}_{t}$, multiplicado por la compuerta de entrada, para decidir qué parte del nuevo estado de la celda candidato se va a almacenar en el estado de la celda actual.

<img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-C.png" alt="lstm7" border="0" width=600>

Finalmente, es necesario decidir cual será la salida. La salida se basará en el estado de la celda, pero será una versión filtrada. Primero se ejecuta una capa sigmoide, para decidir que parte de la celda se va a filtrar. Despues, se ejecuta una capa $tanh$, para crear un vector de valores entre -1 y 1, que se multiplica por la salida de la capa sigmoide, para decidir que valores se van a mostrar como salida.

<img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-o.png" alt="lstm8" border="0" width=600>

Despues de la definición de las LSTM, iniciaremos la implementación. Para utilizar las celdas LSTM, se utilizará `tensorflow.keras.layers.LSTM()`.

<img src="https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/images/08-RNN-architecture-coloured-block-edition.png" alt="lstm9" border="0" width=800>

El modelo tendrá la siguiente estructura:

> Input (text) -> Tokenize -> Embedding -> Layers -> Output (label probability)

La principal diferencia será que se agregará una capa LSTM entre el embedding y la salida. Se creará una nueva capa de embedding para asegurarse de que no se está reutilizando el embedding de la capa anterior. La capa de `text_vectorizer`se puede reutilizar, ya que no se está entrenando.



In [42]:
tf.random.set_seed(42)
from tensorflow.keras import layers
model_2_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_2")

# Create LSTM Model
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_2_embedding(x)
print(x.shape)
x = layers.LSTM(64)(x)
print(x.shape)
outputs=layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

(None, 15, 128)
(None, 64)


In [43]:
# Compile model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [44]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 15, 128)           1280000   
_________________________________________________________________
lstm (LSTM)                  (None, 64)                49408     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
_________________________________________________________________


In [45]:
# Fit model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, 
                                                                     "model_2_LSTM")])

Saving TensorBoard log files to: training_logs/RNN_text/model_2_LSTM/20230225-183636
Epoch 1/5
215/215 [==============================] - 6s 12ms/step - loss: 0.5100 - accuracy: 0.7416 - val_loss: 0.4566 - val_accuracy: 0.7822
Epoch 2/5
215/215 [==============================] - 2s 9ms/step - loss: 0.3176 - accuracy: 0.8717 - val_loss: 0.5138 - val_accuracy: 0.7756
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.2201 - accuracy: 0.9152 - val_loss: 0.5858 - val_accuracy: 0.7677
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.1556 - accuracy: 0.9428 - val_loss: 0.6041 - val_accuracy: 0.7743
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1076 - accuracy: 0.9594 - val_loss: 0.8746 - val_accuracy: 0.7507


In [46]:
# Make predictions on the validation dataset
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs.shape, model_2_pred_probs[:10] # view the first 10

((762, 1),
 array([[0.007126  ],
        [0.78736776],
        [0.9996376 ],
        [0.0567918 ],
        [0.00258219],
        [0.9996238 ],
        [0.9217022 ],
        [0.9997993 ],
        [0.9994954 ],
        [0.664574  ]], dtype=float32))

In [47]:
# Round out predictions and reduce to 1-dimensional array
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [48]:
# Calculate LSTM model results
model_2_results = hf.calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)
model_2_results

{'accuracy': 75.06561679790026,
 'precision': 0.7510077975908164,
 'recall': 0.7506561679790026,
 'f1': 0.7489268622514025}

In [49]:
# Create a helper function to compare our baseline results to new model results
def compare_baseline_to_new_results(baseline_results, new_model_results):
  for key, value in baseline_results.items():
    print(f"Baseline {key}: {value:.2f}, New {key}: {new_model_results[key]:.2f}, Difference: {new_model_results[key]-value:.2f}")

In [50]:
# Compare model 2 to baseline
compare_baseline_to_new_results(baseline_results, model_2_results)

Baseline accuracy: 79.27, New accuracy: 75.07, Difference: -4.20
Baseline precision: 0.81, New precision: 0.75, Difference: -0.06
Baseline recall: 0.79, New recall: 0.75, Difference: -0.04
Baseline f1: 0.79, New f1: 0.75, Difference: -0.04
